# NEON pre-processor notebook

##### Author : Negin Sobhani negins@ucar.edu [@negin513](https://github.com/negin513)

##### Last revised: 2022-10-11
_______

This is a notebook for reading neon simulation and evaluation files and save them as csv files. This would be next used for creating statistics and spatial plots. 

In [1]:
import os
import time
import datetime

import numpy as np
import pandas as pd
import xarray as xr

from glob import glob
from os.path import join

import tqdm
import cftime
import neon_utils as nu

In [2]:
print('xarray '+xr.__version__) ##-- was working with 0.20.0

xarray 2023.6.0


## Dask Cluster
The following will spin up a dask cluster,  only works on NCAR machines. 
16 workers worked best for NEON.

In [3]:
from distributed import Client
from ncar_jobqueue import NCARCluster

cluster = NCARCluster()
cluster.scale(16)
client = Client(cluster)
client

/glade/u/apps/opt/conda/envs/npl-2023b/lib/python3.10/site-packages/dask_jobqueue/core.py:255: FutureWarning: job_extra has been renamed to job_extra_directives. You are still using it (even if only set to []; please also check config files). If you did not set job_extra_directives yet, job_extra will be respected for now, but it will be removed in a future release. If you already set job_extra_directives, job_extra is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
/glade/u/apps/opt/conda/envs/npl-2023b/lib/python3.10/site-packages/dask_jobqueue/core.py:274: FutureWarning: env_extra has been renamed to job_script_prologue. You are still using it (even if only set to []; please also check config files). If you did not set job_script_prologue yet, env_extra will be respected for now, but it will be removed in a future release. If you already set job_script_prologue, env_extra is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
/glade/u/apps/opt/con

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/wwieder/proxy/8787/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/wwieder/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.206.49:42962,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/wwieder/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


---------------------------

In [4]:
# -- read list of sites:
#neon_sites_pft = pd.read_csv('/glade/scratch/wwieder/run_NEON_v2/tools/site_and_regional/neon_sites_dompft.csv')
neon_sites_pft = pd.read_csv('/glade/u/home/wwieder/CTSM/tools/site_and_regional/neon_sites_dompft.csv')
neon_sites = neon_sites_pft['Site'].to_list()

failed_sites = [] # -- list for saving failed sites

### Download evaluation data
This is slow and only has to be done once!

In [5]:
'''
neon_dir = '/glade/work/wwieder/neon/'
if not os.path.exists (neon_dir):
    print ("neon directory does not exist... creating it now!")
    os.mkdir(neon_dir)

eval_dir = '/glade/work/wwieder/neon/eval_files/'
if not os.path.exists (eval_dir):
    print ("eval directory does not exist... creating it now!")
    os.mkdir(eval_dir)
    
for neon_site in neon_sites[1:]:
    nu.download_eval_files(neon_site,eval_dir)
'''

'\nneon_dir = \'/glade/work/wwieder/neon/\'\nif not os.path.exists (neon_dir):\n    print ("neon directory does not exist... creating it now!")\n    os.mkdir(neon_dir)\n\neval_dir = \'/glade/work/wwieder/neon/eval_files/\'\nif not os.path.exists (eval_dir):\n    print ("eval directory does not exist... creating it now!")\n    os.mkdir(eval_dir)\n    \nfor neon_site in neon_sites[1:]:\n    nu.download_eval_files(neon_site,eval_dir)\n'

In [6]:
'''
# Download datm data

atm_dir = '/glade/work/wwieder/neon/atm_files/'
    
for neon_site in neon_sites[1:]:
    nu.download_atm_files(neon_site,atm_dir)
'''

"\n# Download datm data\n\natm_dir = '/glade/work/wwieder/neon/atm_files/'\n    \nfor neon_site in neon_sites[1:]:\n    nu.download_atm_files(neon_site,atm_dir)\n"

In [7]:
'''
# Download CLM history output
hist_dir = '/glade/work/wwieder/neon/hist_files/'
    
#for neon_site in neon_sites[1:]:
#   nu.download_atm_files(neon_site,hist_dir)
'''

"\n# Download CLM history output\nhist_dir = '/glade/work/wwieder/neon/hist_files/'\n    \n#for neon_site in neon_sites[1:]:\n#   nu.download_atm_files(neon_site,hist_dir)\n"

In [8]:
# -- read only these variables from the whole netcdf files

def preprocess (ds):
    '''
    reading selected variables from ctsm outputs
    '''
    variables = ['FCEV', 'FCTR', 'FGEV','FSH','GPP','FSA','FIRA','AR','HR','ELAI']

    ds_new= ds[variables]
    return ds_new

In [9]:
# -- fix timestamp on CTSM files so they can be matched with eval files

def fix_time (ds):
    '''
    fix time formatting with reading multiple cesm files.
    '''
    nsteps = len(ds.time)
    yr0 = ds['time.year'][0].values
    month0 = ds['time.month'][0].values
    day0 = ds['time.day'][0].values

    date = cftime.datetime(yr0,month0,day0).isoformat() 
    ds['time'] = xr.cftime_range(date, periods=nsteps, freq='30min')
    ds['time']= ds['time'].dt.strftime("%Y-%m-%d %H:%M:%S")
    return ds

In [10]:
#-- path to save the csv files:
csv_dir = "/glade/scratch/wwieder/preprocessed_neon_v2/"

#create the directory if it does not exist:
if not os.path.isdir(csv_dir):
    os.mkdir(csv_dir)

In [12]:
years = ["2018","2019","2020","2021"]

for neon_site in neon_sites:
    try: 

        start_site = time.time()

        print ('---------------------------')
        print ("Extracting data for "+neon_site)
        sim_files =[]
        for year in years:
            sim_path = "/glade/campaign/cgd/tss/wwieder/NEON_v2/hist/"+neon_site+".transient/lnd/hist/"
            #sim_path = "/glade/scratch/wwieder/NEON_testV2/archive/"+neon_site+".transient/lnd/hist/"
            #sim_path = "/glade/scratch/negins/neon_v2/tools/site_and_regional/archive/"+neon_site+".transient/lnd/hist/"
            sim_files.extend(sorted(glob(join(sim_path,neon_site+".transient.clm2.h1."+year+"*.nc"))))

        print("All simulation files for all years: [", len(sim_files), "files]")

        start = time.time()

        ds_ctsm = xr.open_mfdataset(sim_files, decode_times=True, combine='by_coords',parallel=True,preprocess=preprocess)
        ds_ctsm = fix_time (ds_ctsm)

        end = time.time()
        print("Reading all simulation files took:", end-start, "s.")

        eval_files = []
        for year in years:
            eval_dir = "/glade/campaign/cgd/tss/wwieder/NEON_v2/eval/"+neon_site
            #eval_dir = "/glade/work/wwieder/neon/eval_files/"+neon_site
            #eval_dir = "/glade/work/negins/neon_scripts/notebooks/evaluation_files_2/"+neon_site
            eval_files.extend(sorted(glob(join(eval_dir,neon_site+"_eval_"+year+"*.nc"))))


        print ("All evaluation files for all years: [", len(eval_files), "files]")

        start = time.time()

        ds_eval = xr.open_mfdataset(eval_files, decode_times=True, combine='by_coords')

        end = time.time()
        print("Reading all observation files took:", end-start, "s.")
        
        #Convert CTSM data to a Pandas Dataframe for easier handling:
        ctsm_vars = ['FCEV', 'FCTR', 'FGEV','FSH','GPP','FSA','FIRA','AR','HR','ELAI']

        df_ctsm = pd.DataFrame({'time':ds_ctsm.time})
        df_ctsm['time'] = pd.to_datetime(df_ctsm['time'],format= '%Y-%m-%d %H:%M:%S' )

        for var in tqdm.tqdm(ctsm_vars):
            sim_var_name = "sim_"+var
            field = np.ravel ( ds_ctsm[var])     
            df_ctsm[sim_var_name]=field
            #-- shift simulation data by one
            df_ctsm[sim_var_name]=df_ctsm[sim_var_name].shift(-1).values


        #Convert NEON data to a Pandas Dataframe for easier handling:
        eval_vars = ['NEE','FSH','EFLX_LH_TOT','GPP','Rnet']

        df_all = pd.DataFrame({'time':ds_eval.time})

        for var in eval_vars:
            field = np.ravel (ds_eval[var])
            df_all[var]=field
        
        # -- merge two pandas dataframe on time
        df_all=df_all.merge(df_ctsm.set_index('time'), on='time', how='left')

        clm_var = 'sim_EFLX_LH_TOT'
        # -- Latent Heat Flux:
        # -- EFLX_LH_TOT = FCEV + FCTR +FGEV
        df_all [clm_var] = df_all['sim_FCEV']+ df_all['sim_FCTR']+ df_all['sim_FGEV']

        clm_var = 'sim_Rnet'
        # -- Net Radiation:
        # -- Rnet = FSA-FIRA
        df_all [clm_var] = df_all ['sim_FSA']-df_all['sim_FIRA']

        clm_var = 'sim_NEE'
        # -- Net Ecosystem Exchange
        # -- NEE = GPP- (AR+HR)
        # -- It seems like the sign are opposite so we calculated negative NEE
        df_all [clm_var] = -(df_all ['sim_GPP']-(df_all['sim_AR']+df_all['sim_HR']))

        # convert NEE units from  umolm-2s-1 to gc/m2/s
        df_all ['NEE']= df_all ['NEE']*(12.01/1000000)
        df_all ['GPP']= df_all ['GPP']*(12.01/1000000)

        # -- convert gc/m2/s to gc/m2/day
        df_all ['NEE']= df_all['NEE']*60*60*24
        df_all ['sim_NEE']= df_all['sim_NEE']*60*60*24

        df_all ['GPP']= df_all['GPP']*60*60*24
        df_all ['sim_GPP']= df_all['sim_GPP']*60*60*24

        #-- extract year, month, day, hour information from time
        df_all['year'] = df_all['time'].dt.year
        df_all['month'] = df_all['time'].dt.month
        df_all['day'] = df_all['time'].dt.day
        df_all['hour'] = df_all['time'].dt.hour

        tzone_shift = ds_eval.TimeDiffUtcLt

        #convert to local time
        df_all['local_time']= df_all['time'] + datetime.timedelta(hours=int(tzone_shift))
        df_all['local_hour'] = df_all['local_time'].dt.hour


        df_all['site']=neon_site

        csv_out = join(csv_dir, "preprocessed_"+neon_site+"_"+year+".csv")
        df_all.to_csv(csv_out,index=False)
        
        end_site = time.time()
        print ("Extracting data for "+neon_site+" took : ", end_site-start_site, "s.")
        print ("Preprocessed file is saved as : ", csv_out)

    except Exception as e: 
        print (e)
        print ('THIS SITE FAILED:', neon_site)
        failed_sites.append(neon_site)
        pass

---------------------------
Extracting data for BART
All simulation files for all years: [ 1461 files]


Exception ignored in: <bound method GCDiagnosis._gc_callback of <distributed.utils_perf.GCDiagnosis object at 0x2af2d127ed10>>
Traceback (most recent call last):
  File "/glade/u/apps/opt/conda/envs/npl-2023b/lib/python3.10/site-packages/distributed/utils_perf.py", line 183, in _gc_callback
    self._fractional_timer.start_timing()
  File "/glade/u/apps/opt/conda/envs/npl-2023b/lib/python3.10/site-packages/distributed/utils_perf.py", line 118, in start_timing
    assert self._cur_start is None
AssertionError: 


Reading all simulation files took: 10.842947721481323 s.
All evaluation files for all years: [ 48 files]
Reading all observation files took: 3.4494359493255615 s.


100%|██████████| 10/10 [00:29<00:00,  2.92s/it]


Extracting data for BART took :  45.34335160255432 s.
Preprocessed file is saved as :  /glade/scratch/wwieder/preprocessed_neon_v2/preprocessed_BART_2021.csv
---------------------------
Extracting data for HARV
All simulation files for all years: [ 1461 files]
Reading all simulation files took: 11.81951642036438 s.
All evaluation files for all years: [ 48 files]
Reading all observation files took: 3.1807057857513428 s.


...

Extracting data for BONA took :  48.90095663070679 s.
Preprocessed file is saved as :  /glade/scratch/wwieder/preprocessed_neon_v2/preprocessed_BONA_2021.csv
---------------------------
Extracting data for DEJU
All simulation files for all years: [ 1461 files]
Reading all simulation files took: 13.175875425338745 s.
All evaluation files for all years: [ 48 files]
Reading all observation files took: 1.8336660861968994 s.


100%|██████████| 10/10 [00:33<00:00,  3.38s/it]


Extracting data for DEJU took :  50.97748684883118 s.
Preprocessed file is saved as :  /glade/scratch/wwieder/preprocessed_neon_v2/preprocessed_DEJU_2021.csv
---------------------------
Extracting data for HEAL
All simulation files for all years: [ 1461 files]
Reading all simulation files took: 14.005837202072144 s.
All evaluation files for all years: [ 48 files]
Reading all observation files took: 1.752248764038086 s.


100%|██████████| 10/10 [00:32<00:00,  3.21s/it]


Extracting data for HEAL took :  49.67314147949219 s.
Preprocessed file is saved as :  /glade/scratch/wwieder/preprocessed_neon_v2/preprocessed_HEAL_2021.csv
---------------------------
Extracting data for PUUM
All simulation files for all years: [ 0 files]
no files to open
THIS SITE FAILED: PUUM


In [9]:
print ("Extracting data for ", len(failed_sites), "sites failed : ")
print (*failed_sites, sep=" \n")

Extracting data for  2 sites failed : 
NIWO 
PUUM
